In [74]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import pandas as pd
import numpy as np 
import openpyxl
import sklearn.linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier  
from sklearn.ensemble import RandomForestRegressor  
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import svm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import re


url = 'https://fbref.com/en/comps/13/schedule/Ligue-1-Scores-and-Fixtures'
page = requests.get(url)
html_content = page.content
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find('table')
df = pd.read_html(str(table))[0].dropna(subset = 'Wk').reset_index(drop=True)
df['Wk'] = df['Wk'].astype(int)

affiches = df[df['Score'].isna()]
first = affiches['Wk'].min()


df = df[df['Wk'] <= first]
df = df[['Wk', 'Home', 'Score', 'Away']]
noms_colonnes = ['Journée','Domicile','Extérieur','Buts domicile','Buts extérieur','Résultat']
df['Buts domicile'] = df['Score'].apply(lambda x: int(str(x)[0]) if str(x)[0].isdigit() else 0)
df['Buts extérieur'] = df['Score'].apply(lambda x: int(str(x)[2]) if str(x)[2].isdigit() else 0)

df.drop(['Score'], axis = 1, inplace = True)
df['Résultat'] = -1
df.loc[df['Buts domicile'] > df['Buts extérieur'], 'Résultat'] = 1
df.loc[df['Buts domicile'] == df['Buts extérieur'], 'Résultat'] = 0
df.columns = noms_colonnes

url = "https://www.deux-zero.com/ligue-1/classement-general/edition/2023-2024/init/1/fin/"
df_classement = pd.DataFrame()
annee = re.search(r'\d{4}-\d{4}', url).group()
for i in range(1,first+1):
    url2 = url + str(i)
    page = requests.get(url2)
    soup = BeautifulSoup(page.text, "html.parser")

    tableau = soup.find("table", class_="tableau")
    equipes = tableau.select('tr.classement td.gras, tr.classement-avec-separateur td.gras')
    equipes_final = []
    for equipe in equipes:
        equipes_final.append(equipe.get_text(strip=True))

    classements = tableau.select('tr.classement span.flag-place, tr.classement-avec-separateur span.flag-place')
    classement_final = []
    for classement in classements:
        if classement.get_text(strip=True) == "-":
            classement_final.append(classement_final[-1])
        else:
            classement_final.append(classement.get_text(strip=True))
    dictionnaire_classement = dict(zip(equipes_final, classement_final))
    if i==1:
        df_classement["Équipes"] = equipes_final
        df_classement["J" + str(i)] = df_classement['Équipes'].map(dictionnaire_classement)
    else:
        df_classement["J" + str(i)] = df_classement['Équipes'].map(dictionnaire_classement)


glossaire = {'AS Monaco FC' : 'Monaco', 'FC Nantes Atlantique' : 'Nantes', 
'Havre AC' : 'Le Havre', 'Montpellier Hérault SC' : 'Montpellier', 'Paris Saint-Germain FC' : 'Paris S-G',  
 'Olympique Lyonnais' : 'Lyon', 'RC Lens' : 'Lens', 'RC Strasbourg' : 'Strasbourg',
 'RC Strasbourg Alsace' : 'Strasbourg', 'OGC Nice' : 'Nice', 
 'Stade Rennais FC' : 'Rennes', 'Olympique de Marseille' : 'Marseille', 'Lille OSC' : 'Lille', 
 'Toulouse FC' : 'Toulouse','FC Metz' : 'Metz', 'FC Lorient' : 'Lorient', 'Stade Brestois 29' : 'Brest',
  'FC Nantes' : 'Nantes', 'Clermont Foot 63' : 'Clermont Foot', 'Stade de Reims' : 'Reims'}

df_classement['Équipes'] = df_classement['Équipes'].replace(glossaire)


df_merge1 = pd.merge(df, df_classement, left_on=['Domicile'], right_on=['Équipes'], how='left')
conditions = [df_merge1['Journée'] == (i + 1) for i in range(1, max(df_merge1['Journée']) + 1)]
valeurs = [df_merge1[f'J{i}'] for i in range(1, max(df_merge1['Journée']) + 1)]
df['Classement D'] = np.select(conditions, valeurs)
df['Classement D'] = df['Classement D'].replace({0: np.nan}).astype(pd.Int64Dtype())



df_merge2 = pd.merge(df, df_classement, left_on=['Extérieur'], right_on=['Équipes'], how='left')
conditions = [df_merge2['Journée'] == (i + 1) for i in range(1, max(df_merge2['Journée']) + 1)]
valeurs = [df_merge2[f'J{i}'] for i in range(1, max(df_merge2['Journée']) + 1)]
df['Classement E'] = np.select(conditions, valeurs)
df['Classement E'] = df['Classement E'].replace({0: np.nan}).astype(pd.Int64Dtype())

df.insert(3, 'Equipe 1 à Domicile', 1)
noms_colonnes = ['Journée', 'Equipe 1', 'Equipe 2', 'Equipe 1 à Domicile', 'Buts Equipe 1', 'Buts Equipe 2', 'Résultat', 'Classement Equipe 1', 'Classement Equipe 2']
df.columns = noms_colonnes
df_copie = df.copy()[['Journée', 'Equipe 2', 'Equipe 1', 'Equipe 1 à Domicile', 'Buts Equipe 2', 'Buts Equipe 1', 'Résultat', 'Classement Equipe 2', 'Classement Equipe 1']]
df_copie.columns = noms_colonnes
df_copie['Equipe 1 à Domicile'] = 0
df = df._append(df_copie, ignore_index=True)
df = df.sort_values(by=['Equipe 1', 'Journée']).reset_index(drop=True)
conditions = [
    (df['Buts Equipe 1'] > df['Buts Equipe 2']),
    (df['Buts Equipe 1'] < df['Buts Equipe 2'])
]
valeurs = [1, -1]
df['Résultat'] = 0
df['Résultat'] = np.select(conditions, valeurs)


df['Moyenne_BM par 1'] = (df.groupby(['Equipe 1'])['Buts Equipe 1'].cumsum() - df['Buts Equipe 1']) / (df.groupby(['Equipe 1'])['Journée'].cumcount())
df['Moyenne_BM par 2'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Moyenne_BM par 1_y']
df['Moyenne_BM par 1 selon Equipe 1 à Domicile'] = (df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Buts Equipe 1'].cumsum() - df['Buts Equipe 1']) / (df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Journée'].cumcount())
df['Moyenne_BM par 2 selon Equipe 1 à Domicile'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Moyenne_BM par 1 selon Equipe 1 à Domicile_y']
df['Moyenne_BE par 1'] = (df.groupby(['Equipe 1'])['Buts Equipe 2'].cumsum() - df['Buts Equipe 2']) / (df.groupby(['Equipe 1'])['Journée'].cumcount())
df['Moyenne_BE par 2'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Moyenne_BE par 1_y']
df['Moyenne_BE par 1 selon Equipe 1 à Domicile'] = (df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Buts Equipe 2'].cumsum() - df['Buts Equipe 2']) / (df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Journée'].cumcount())
df['Moyenne_BE par 2 selon Equipe 1 à Domicile'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Moyenne_BE par 1 selon Equipe 1 à Domicile_y']
df['Forme 1'] = df.groupby(['Equipe 1'])['Résultat'].rolling(window=6, min_periods=2).sum().reset_index(drop=True)-df['Résultat']
df['Forme 2'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Forme 1_y']
df2 = pd.read_csv('C:\\Users\\vtgra\\Desktop\\Projet python\\dataframe_final.csv',encoding = 'utf-8')
df['Historique'] = pd.merge(df,df2.groupby(['Equipe 1', 'Equipe 2'])['Résultat'].sum(), on =[ 'Equipe 1', 'Equipe 2'], how='left')['Résultat_y']

df_reg = df.copy()
df = df[df['Journée']!=first]
df.head(10)




KeyError: False

In [91]:

dataframe_regression = df2.dropna().copy()

X_train = dataframe_regression[["Equipe 1 à Domicile","Classement Equipe 1",  "Classement Equipe 2",  "Moyenne_BM par 1", "Moyenne_BM par 1 selon Equipe 1 à Domicile", "Moyenne_BE par 1", "Moyenne_BE par 1 selon Equipe 1 à Domicile","Moyenne_BM par 2", "Moyenne_BM par 2 selon Equipe 1 à Domicile", "Moyenne_BE par 2", "Moyenne_BE par 2 selon Equipe 1 à Domicile", 'Forme 1', 'Forme 2', 'Historique']]
Y_train = dataframe_regression["Buts Equipe 1"]


X_train2 = dataframe_regression[["Equipe 1 à Domicile", "Classement Equipe 1",  "Classement Equipe 2",  "Moyenne_BM par 1", "Moyenne_BM par 1 selon Equipe 1 à Domicile", "Moyenne_BE par 1", "Moyenne_BE par 1 selon Equipe 1 à Domicile","Moyenne_BM par 2", "Moyenne_BM par 2 selon Equipe 1 à Domicile", "Moyenne_BE par 2", "Moyenne_BE par 2 selon Equipe 1 à Domicile", 'Forme 1', 'Forme 2', 'Historique']]
Y_train2 = dataframe_regression["Résultat"]


random_forest_model = RandomForestClassifier(n_estimators=1000, random_state=5)
random_forest_model.fit(X_train2, Y_train2)

random_forest_model2 = RandomForestClassifier(n_estimators=1000, random_state=5)
random_forest_model.fit(X_train, Y_train)


model=LinearRegression()
model.fit(X_train,Y_train)


LinearRegression()

In [107]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import os

df_int = df.copy()
df_int['Résultat'] = df_int['Résultat'].replace({1: 'V', 0: 'N', -1: 'D'})
def trouver_chemins_images_avec_mot_cle(dossier, mot_cle):
    chemins_images = []

    fichiers = os.listdir(dossier)

    for fichier in fichiers:
        chemin_fichier = os.path.join(dossier, fichier)

        if os.path.isfile(chemin_fichier) and fichier.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            
            if mot_cle.lower() in fichier.lower():
                chemins_images.append(chemin_fichier)
                print(f"Chemin de l'image trouvée : {chemin_fichier}")

    return chemins_images[0]

photo1 = None
photo2 = None
first = affiches['Wk'].min()

def fonction_journées(event):
    sélection = int(journées_combobox.get()[1:])
    matchs = [affiche for affiche, wk in zip(affiches['Home'] + ' - ' + affiches['Away'], affiches['Wk']) if wk == sélection]
    matchs_combobox['values'] = matchs
    matchs_combobox.set("Sélectionnez un match")

def fonction_stats(event):
    global photo1, photo2, equipe1, equipe2
    equipe1 = matchs_combobox.get().split(' - ')[0].strip()
    equipe2 = matchs_combobox.get().split(' - ')[1].strip()
    noms_colonnes = ['Classement', 'Moyenne buts marqués', 'Moyenne buts encaissés', 'Forme', 'Historique']
    df_match1 = df[((df['Equipe 1'] == equipe1) & (df['Journée'] == first - 1))][['Classement Equipe 1', 'Moyenne_BM par 1','Moyenne_BE par 1']].reset_index(drop=True)
    df_match2 = df[((df['Equipe 2'] == equipe2) & (df['Journée'] == first - 1))][[ 'Classement Equipe 2', 'Moyenne_BM par 2', 'Moyenne_BE par 2']].reset_index(drop=True)
    df_match1['Moyenne_BM par 1'] = df_match1['Moyenne_BM par 1'].round(2)
    df_match1['Moyenne_BE par 1'] = df_match1['Moyenne_BE par 1'].round(2)
    df_match2['Moyenne_BM par 2'] = df_match2['Moyenne_BM par 2'].round(2)
    df_match2['Moyenne_BE par 2'] = df_match2['Moyenne_BE par 2'].round(2)
    df_match1['Forme'] = str(df_int[((df_int['Equipe 1'] == equipe1) & (df_int['Journée'] == first - 1))]['Résultat'].iloc[0]) + str(df_int[((df_int['Equipe 1'] == equipe1) & (df_int['Journée'] == first - 2))]['Résultat'].iloc[0]) + str(df_int[((df_int['Equipe 1'] == equipe1) & (df_int['Journée'] == first - 3))]['Résultat'].iloc[0]) + str(df_int[((df['Equipe 1'] == equipe1) & (df_int['Journée'] == first - 4))]['Résultat'].iloc[0]) + str(df_int[((df_int['Equipe 1'] == equipe1) & (df_int['Journée'] == first - 5))]['Résultat'].iloc[0])
    df_match2['Forme'] = str(df_int[((df_int['Equipe 1'] == equipe2) & (df_int['Journée'] == first - 1))]['Résultat'].iloc[0]) + str(df_int[((df_int['Equipe 1'] == equipe2) & (df_int['Journée'] == first - 2))]['Résultat'].iloc[0]) + str(df_int[((df_int['Equipe 1'] == equipe2) & (df_int['Journée'] == first - 3))]['Résultat'].iloc[0]) + str(df_int[((df['Equipe 1'] == equipe2) & (df_int['Journée'] == first - 4))]['Résultat'].iloc[0]) + str(df_int[((df_int['Equipe 1'] == equipe2) & (df_int['Journée'] == first - 5))]['Résultat'].iloc[0])
    try:
        a = str(int(df2[(df2['Equipe 1'] == equipe1) & (df2['Equipe 2'] == equipe2)]['Résultat'].value_counts().add(df[(df['Equipe 1'] == equipe1) & (df['Equipe 2'] == equipe2)]['Résultat'].value_counts(),fill_value=0)[1]))
    except Exception as e:
        a = '0'
    try:
        b = str(int(df2[(df2['Equipe 1'] == equipe1) & (df2['Equipe 2'] == equipe2)]['Résultat'].value_counts().add(df[(df['Equipe 1'] == equipe1) & (df['Equipe 2'] == equipe2)]['Résultat'].value_counts(),fill_value=0)[-1]))
    except Exception as e:
        b = '0'
    try:
        c = str(int(df2[(df2['Equipe 1'] == equipe1) & (df2['Equipe 2'] == equipe2)]['Résultat'].value_counts().add(df[(df['Equipe 1'] == equipe1) & (df['Equipe 2'] == equipe2)]['Résultat'].value_counts(),fill_value=0)[0]))
    except Exception as e:
        c = '0'
    df_match1['Historique'] = a + 'V ' + b + 'D ' + c + 'N'
    df_match2['Historique'] = b + 'V ' + a + 'D ' + c + 'N'

    df_match1.columns = noms_colonnes
    df_match2.columns = noms_colonnes

    for i, (col_name, value) in enumerate(df_match1.iloc[0].items()):
        if col_name == 'Classement':
            value = int(value)
        label_title = tk.Label(frame_tableau1, text=f"{col_name}:", padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=20)
        label_title.grid(row=i+3, column=0, sticky="w")

        label_value = tk.Label(frame_tableau1, text=value, padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=20)
        label_value.grid(row=i+3, column=1, sticky="w")
    
    for i, (col_name, value) in enumerate(df_match2.iloc[0].items()):
        if col_name == 'Classement':
            value = int(value)
        label_title = tk.Label(frame_tableau2, text=f"{col_name}:", padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=20)
        label_title.grid(row=i+3, column=0, sticky="w")

        label_value = tk.Label(frame_tableau2, text=value, padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=20)
        label_value.grid(row=i+3, column=1, sticky="w")

        image_path1 = trouver_chemins_images_avec_mot_cle('C:\\Users\\vtgra\\Desktop\\Projet python', equipe1)
        image1 = Image.open(image_path1)
        image1 = image1.resize((100, 120), Image.ANTIALIAS)  
        photo1 = ImageTk.PhotoImage(image1)

        label_image1 = tk.Label(frame_tableau1, image=photo1)
        label_image1.image = photo1  
        label_image1.grid(row=0, column=0, columnspan=2)

        image_path2 = trouver_chemins_images_avec_mot_cle('C:\\Users\\vtgra\\Desktop\\Projet python', equipe2)
        image2 = Image.open(image_path2)
        image2 = image2.resize((100, 120), Image.ANTIALIAS)  
        photo2 = ImageTk.PhotoImage(image2)

        label_image2 = tk.Label(frame_tableau2, image=photo2)
        label_image2.image = photo2  
        label_image2.grid(row=0, column=0, columnspan=2)

        bouton_predictions = tk.Button(window, text="Prédictions", command=lambda: fonction_predictions(event = None))
        bouton_predictions.place(relx=0.5, rely=0.8, anchor="center")


def fonction_predictions(event):
    global photo1, photo2, equipe1, equipe2

    new_window = tk.Toplevel()
    new_window.attributes('-fullscreen', True)
    titre_label = tk.Label(new_window, text="Prédiction pour le match " + equipe1 + " - " + equipe2, font=("Helvetica", 24, "bold"))
    titre_label.pack(pady=150)

    frame_tableau3 = tk.Frame(new_window)
    frame_tableau3.pack(expand=True, anchor="w")
    frame_tableau4 = tk.Frame(new_window)
    frame_tableau4.pack(expand=True, anchor="w")


    label_image1 = tk.Label(frame_tableau3, image=photo1)
    label_image1.image = photo1  
    label_image1.grid(row=0, column=0, columnspan=2)

    label_image2 = tk.Label(frame_tableau4, image=photo2)
    label_image2.image = photo2  
    label_image2.grid(row=0, column=0, columnspan=2)

    frame_tableau3.place(in_=new_window, anchor="w", relx=0.37, rely=0.4)
    frame_tableau4.place(in_=new_window, anchor="w", relx=0.57, rely=0.4)

    try:
        match_aller = df2[((df2['Equipe 1'] == equipe1) & (df2['Equipe 2'] == equipe2))][ "Historique"].iloc[0]
    except Exception as e:
        match_aller = 0
    X_test = df_reg[((df_reg['Equipe 1'] == equipe1) & (df_reg['Journée'] == first))][[ "Classement Equipe 1",  "Classement Equipe 2",  "Moyenne_BM par 1", "Moyenne_BM par 1 selon Equipe 1 à Domicile", "Moyenne_BE par 1", "Moyenne_BE par 1 selon Equipe 1 à Domicile","Moyenne_BM par 2", "Moyenne_BM par 2 selon Equipe 1 à Domicile", "Moyenne_BE par 2", "Moyenne_BE par 2 selon Equipe 1 à Domicile", 'Forme 1', 'Forme 2']]
    X_test = X_test.reset_index(drop=True)
    X_test[["Classement Equipe 2","Moyenne_BM par 2", "Moyenne_BM par 2 selon Equipe 1 à Domicile", "Moyenne_BE par 2", "Moyenne_BE par 2 selon Equipe 1 à Domicile", 'Forme 2']] = df_reg[((df_reg['Equipe 2'] == equipe2) & (df_reg['Journée'] == first))][["Classement Equipe 2","Moyenne_BM par 2", "Moyenne_BM par 2 selon Equipe 1 à Domicile", "Moyenne_BE par 2", "Moyenne_BE par 2 selon Equipe 1 à Domicile", 'Forme 2']].reset_index(drop=True)
    X_test.insert(0, 'Equipe 1 à Domicile', 1)
    X_test['Historique'] = df2[((df2['Equipe 1'] == equipe1) & (df2['Equipe 2'] == equipe2))][ "Historique"].iloc[-1] + match_aller
    X_test2 = df_reg[((df_reg['Equipe 1'] == equipe2) & (df_reg['Journée'] == first))][[ "Classement Equipe 1",  "Classement Equipe 2",  "Moyenne_BM par 1", "Moyenne_BM par 1 selon Equipe 1 à Domicile", "Moyenne_BE par 1", "Moyenne_BE par 1 selon Equipe 1 à Domicile","Moyenne_BM par 2", "Moyenne_BM par 2 selon Equipe 1 à Domicile", "Moyenne_BE par 2", "Moyenne_BE par 2 selon Equipe 1 à Domicile", 'Forme 1', 'Forme 2']]
    X_test2 = X_test2.reset_index(drop=True)
    X_test2[["Classement Equipe 2","Moyenne_BM par 2", "Moyenne_BM par 2 selon Equipe 1 à Domicile", "Moyenne_BE par 2", "Moyenne_BE par 2 selon Equipe 1 à Domicile", 'Forme 2']]= df_reg[((df_reg['Equipe 2'] == equipe1) & (df_reg['Journée'] == first))][["Classement Equipe 2","Moyenne_BM par 2", "Moyenne_BM par 2 selon Equipe 1 à Domicile", "Moyenne_BE par 2", "Moyenne_BE par 2 selon Equipe 1 à Domicile", 'Forme 2']].reset_index(drop=True)
    X_test2.insert(0, 'Equipe 1 à Domicile', 0)
    X_test2['Historique'] = df2[((df2['Equipe 1'] == equipe2) & (df2['Equipe 2'] == equipe1))][ "Historique"].iloc[-1] - match_aller
    X_test3 = df_reg[((df_reg['Equipe 1'] == equipe1) & (df_reg['Journée'] == first))][[ "Classement Equipe 1",  "Classement Equipe 2",  "Moyenne_BM par 1", "Moyenne_BM par 1 selon Equipe 1 à Domicile", "Moyenne_BE par 1", "Moyenne_BE par 1 selon Equipe 1 à Domicile","Moyenne_BM par 2", "Moyenne_BM par 2 selon Equipe 1 à Domicile", "Moyenne_BE par 2", "Moyenne_BE par 2 selon Equipe 1 à Domicile", 'Forme 1', 'Forme 2']]
    X_test3 = X_test3.reset_index(drop=True)
    X_test3[["Classement Equipe 2","Moyenne_BM par 2", "Moyenne_BM par 2 selon Equipe 1 à Domicile", "Moyenne_BE par 2", "Moyenne_BE par 2 selon Equipe 1 à Domicile", 'Forme 2']]= df_reg[((df_reg['Equipe 2'] == equipe2) & (df_reg['Journée'] == first))][["Classement Equipe 2","Moyenne_BM par 2", "Moyenne_BM par 2 selon Equipe 1 à Domicile", "Moyenne_BE par 2", "Moyenne_BE par 2 selon Equipe 1 à Domicile", 'Forme 2']].reset_index(drop=True)
    X_test3.insert(0, 'Equipe 1 à Domicile', 1)
    X_test3['Historique'] = df2[((df2['Equipe 1'] == equipe1) & (df2['Equipe 2'] == equipe2))][ "Historique"].iloc[-1] + match_aller


    y_pred3 = random_forest_model.predict(X_test3)

    probabilites_buts1 = random_forest_model.predict_proba(X_test)
    probabilites_buts2 = random_forest_model.predict_proba(X_test2)
    
    y_pred1 =(model.predict(X_test))
    y_pred2 =(model.predict(X_test2))

    print(y_pred1, y_pred2, y_pred3)

    if y_pred3 == 1:
        while round(y_pred1[0]) - round(y_pred2[0]) < y_pred3:
            y_pred1[0] = y_pred1[0] + 1
    elif y_pred3 == -1:
        while round(y_pred2[0]) - round(y_pred1[0]) < abs(y_pred3):
            y_pred2[0] = y_pred2[0] + 1
    else:
        if round(y_pred1[0]) - round(y_pred2[0]) < y_pred3:
            y_pred1[0] = y_pred1[0] + 1
        elif round(y_pred2[0]) - round(y_pred1[0]) < y_pred3:
            y_pred2[0] = y_pred2[0] + 1

    probabilites_buts1 = list(enumerate(probabilites_buts1[0]))
    tableau1 = ttk.Treeview(new_window, columns=("nombres de buts", "proba"), show="headings")

    tableau1.heading("nombres de buts", text="Nombres de Buts",  anchor="w")
    tableau1.heading("proba", text="Probabilité",  anchor="w")

    for k in probabilites_buts1:
        tableau1.insert("", "end", values=k)
    
    tableau1.column("proba", width=100)
    tableau1.column("nombres de buts", width=100)

    probabilites_buts2 = list(enumerate(probabilites_buts2[0]))
    tableau2 = ttk.Treeview(new_window, columns=("nombres de buts", "proba"), show="headings")

    tableau2.heading("nombres de buts", text="Nombres de Buts",  anchor="w")
    tableau2.heading("proba", text="Probabilité",  anchor="w")

    for k in probabilites_buts2:
        tableau2.insert("", "end", values=k)

    tableau2.column("proba", width=100)
    tableau2.column("nombres de buts", width=100)
   
    tableau1.place(in_=new_window, anchor="w", relx=0.35, rely=0.6)
    tableau2.place(in_=new_window, anchor="w", relx=0.55, rely=0.6)

    score_1 = tk.Label(new_window, text = str(round(y_pred1[0])), font=("Helvetica", 24, "bold"))
    score_1.place(in_=new_window, anchor="w", relx=0.47, rely=0.4)
    
    score_2 = tk.Label(new_window, text = str(round(y_pred2[0])), font=("Helvetica", 24, "bold"))
    score_2.place(in_=new_window, anchor="w", relx=0.52, rely=0.4)

    tiret = tk.Label(new_window, text = '-', font=("Helvetica", 24, "bold"))
    tiret.place(in_=new_window, anchor="w", relx=0.49, rely=0.4)



    

window = tk.Tk()
window.title("Sélection de la Journée")
window.attributes('-fullscreen', True)


image_path_intro = "C:\\Users\\vtgra\\Desktop\\Projet python\\Logo_Ligue_1_Uber_Eats_2022.png"
image_intro = Image.open(image_path_intro)
image_intro = image_intro.resize((130, 180), Image.ANTIALIAS)
photo_intro = ImageTk.PhotoImage(image_intro)



frame_intro = tk.Frame(window)
frame_intro.pack(expand=True, anchor="w")

label_image_intro = tk.Label(frame_intro, image=photo_intro)
label_image_intro.image = photo_intro
label_image_intro.grid(row=0, column=0)


frame_menus = tk.Frame(window)
frame_menus.pack(expand=True, anchor="w")

label_journee = tk.Label(frame_menus, text="Choisissez une journée:")
label_journee.grid(row=0, column=0, padx=10, pady=10)

journées = [f"J{i}" for i in range(first, 35)]
journées_combobox = ttk.Combobox(frame_menus, values=journées, state="readonly")
journées_combobox.set("Sélectionnez une journée")
journées_combobox.grid(row=0, column=1, padx=10, pady=10)
journées_combobox.bind("<<ComboboxSelected>>", fonction_journées)

label_match = tk.Label(frame_menus, text="Choisissez un match:")
label_match.grid(row=1, column=0, padx=10, pady=10)

matchs_combobox = ttk.Combobox(frame_menus, state="readonly")
matchs_combobox.set("Sélectionnez un match")
matchs_combobox.grid(row=1, column=1, padx=10, pady=10)
matchs_combobox.bind("<<ComboboxSelected>>", fonction_stats)


frame_tableau1 = tk.Frame(window)
frame_tableau1.pack(expand=True, anchor="w")
frame_tableau2 = tk.Frame(window)
frame_tableau2.pack(expand=True, anchor="w")



label_texte = tk.Label(frame_tableau1, text="")
label_texte.grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="w")
label_texte = tk.Label(frame_tableau2, text="")
label_texte.grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="w")

window.update_idletasks()
frame_intro.place(in_=window, anchor="w", relx=0.47, rely=0.1)
frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.25)
frame_tableau1.place(in_=window, anchor="w", relx=0.17, rely=0.4)
frame_tableau2.place(in_=window, anchor="w", relx=0.67, rely=0.4)




window.mainloop()








C:\Users\vtgra\AppData\Local\Temp\ipykernel_34284\2448536591.py:220: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image_intro = image_intro.resize((130, 180), Image.ANTIALIAS)


Chemin de l'image trouvée : C:\Users\vtgra\Desktop\Projet python\Logo_Clermont Foot_63_depuis_2020.png
Chemin de l'image trouvée : C:\Users\vtgra\Desktop\Projet python\Logo_Olympique_Lyonnais_-_2022.png
Chemin de l'image trouvée : C:\Users\vtgra\Desktop\Projet python\Logo_Clermont Foot_63_depuis_2020.png
Chemin de l'image trouvée : C:\Users\vtgra\Desktop\Projet python\Logo_Olympique_Lyonnais_-_2022.png
Chemin de l'image trouvée : C:\Users\vtgra\Desktop\Projet python\Logo_Clermont Foot_63_depuis_2020.png
Chemin de l'image trouvée : C:\Users\vtgra\Desktop\Projet python\Logo_Olympique_Lyonnais_-_2022.png
Chemin de l'image trouvée : C:\Users\vtgra\Desktop\Projet python\Logo_Clermont Foot_63_depuis_2020.png
Chemin de l'image trouvée : C:\Users\vtgra\Desktop\Projet python\Logo_Olympique_Lyonnais_-_2022.png
Chemin de l'image trouvée : C:\Users\vtgra\Desktop\Projet python\Logo_Clermont Foot_63_depuis_2020.png
Chemin de l'image trouvée : C:\Users\vtgra\Desktop\Projet python\Logo_Olympique_Lyonn

C:\Users\vtgra\AppData\Local\Temp\ipykernel_34284\2448536591.py:85: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image1 = image1.resize((100, 120), Image.ANTIALIAS)
C:\Users\vtgra\AppData\Local\Temp\ipykernel_34284\2448536591.py:94: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image2 = image2.resize((100, 120), Image.ANTIALIAS)


[1.33282999] [0.99623831] [2]


   Classement Equipe 2  Moyenne_BM par 2  \
0                    5          1.266667   

   Moyenne_BM par 2 selon Equipe 1 à Domicile  Moyenne_BE par 2  \
0                                         1.0               1.0   

   Moyenne_BE par 2 selon Equipe 1 à Domicile  Forme 2  
0                                         1.0      2.0  
